In [ ]:
##############  Grads to Geotiff  Script   #################################
# Copyright Ryan L. Crumley, 2019
# ryanlcrumley@gmail.com

# Description: This script takes SnowModel output in the grads format, and 
# writes a geotiff for the timeslices/model iteration of interest. It uses numpy arrays and 
# GDAL to create the Geotiff. Example gdat files from a recent study in the Chugach
# mountains are located in this folder, along with example geotiff outputs and a QGIS map
# of the geotiff outputs.

# Import all of the python packages used in this workflow.
import scipy
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from pylab import *
import pandas as pd
import numpy as np
import gdal
import osr

print('All packages imported successfully')

############  USER INPUT  ######################## 
# Define some things for use in the script.
# Which water year of SnowModel output are you accessing?
year = '2011-2016'

# The filename or path to the correct grads file.
input_filename = 'swed.gdat' 

# Which simulation is it from
rank = 'calibration'

# Define the grid size in x and y values
nx = 80
ny = 80

In [ ]:
########### GRADS_TO_ARRAY Function ########################## 

# This function reads in a timeslice of data directly from the SnowModel output grads files.

def grads_to_array(inputfilename,nx,ny,timeslice):
    
    # Open the grads file as 'read binary' using 'rb'
    grads_data = open(inputfilename,'rb')
    
    # The 4 is important here, its serving a similar function as in Glen's scripts.
    grads_data.seek(4*timeslice*nx*ny)
    
    # Create a numpy array from the dataset.
    numpy_data = np.fromfile(grads_data,dtype='float32',count=nx*ny)
    
    # Reshape the numpy array so that it has the correct spatial representation.
    numpy_data = np.reshape(numpy_data,(ny,nx))
    
    # Close the grads file.
    grads_data.close()
    
    # Give me the results of the numpy array
    return numpy_data

print('Readslice is ready')

In [ ]:
############# ARRAY_TO_GEOTIFF Function #######################

# This function takes the data from a single numpy array and turns it into a geotiff.
# The numpy arrays get used later.

def array_to_geotiff(filename,width,height,numpy_array):
    
    ######## USER INPUT SECTION ##################
    # Define some things about your model domain
    upper_left_x = 445036
    x_resolution = 30
    x_skew = 0
    upper_left_y = 1272435
    y_skew = 0
    y_resolution = 30
    #################################################
    
    # Reference the correct geotiff driver in GDAL
    driver = gdal.GetDriverByName("GTiff")
    
    # Begin to set the correct spatial reference to the data
    datasetSRS = osr.SpatialReference()
    
    ##############  USER INPUT SECTION   ############
    # Define the CRS using the EPSG code of your choice
    datasetSRS.ImportFromEPSG(3338)
    ##################################################
    
    # This creates the initial raster dataset. The 4th argument (1) is the number of bands in the input dataset.
    dataset = driver.Create(filename, width, height, 1, gdal.GDT_Float32)
    
    # Set the projection give the geotiff the correct spatial location as previously defined 
    dataset.SetGeoTransform((upper_left_x, x_resolution, x_skew, upper_left_y, y_skew, y_resolution))
    dataset.SetProjection(datasetSRS.ExportToWkt())
    
    # Create the output dataset from with one band
    outband = dataset.GetRasterBand(1)
    # This is the numpy array from the input dataset 
    outband.WriteArray(numpy_array)
    # This took me a bit to figure out, but there needs to be a Flush Cache call here in order for this to work,
    outband.FlushCache()  
    
    
print('Array_to_Geotiff is ready')

In [ ]:
# For loop to make a single geotiff for each day of the water year.

print('all packages imported successfully')
print('Grads_to_Array is ready for action')
print('Array_to_Geotiff is ready for action')
print('working on geotiff for loop......')

############ USER INPUT SECTION ##########################
# Make a range of numbers that corresponds to the timeslices / date ranges that you care about.
# If day 220 is the model day/iteration you want to access, begin timeframe at 219 because python 
# range function always begins at 0. The second number in timeframe is the last model day you want to access.
# For this example gdate file, the iteration timeframe goes from Sep 1st 2011 through Aug 31st 2016, so the possible
# iterations to print are 1827.

timeframe = range(219,225)
############################################################

# Run the for loop, chooses the input files, timeslices and links to the functions
for i in timeframe:
    data_array = grads_to_array(input_filename,nx,ny,i)
    output_filename = 'geotiffs/swed_'+rank+'_'+year+'_iteration'+str(i+1)+'.tif'
    array_to_geotiff(output_filename,nx,ny,data_array)
    print('....working on timeframe '+year+' iteration '+str(i+1))
    
    